In [ ]:
import os
from pathlib import Path
from astrocast.analysis import IO, Video, Events, Plotting

astroPlot = Plotting(None)

Here, we will showcase how to run the astrocast pipeline on a small sample dataset. 

# 0. Explore the sample data

In [ ]:
sample_path = Path("../testdata/bigger_data.h5")
assert sample_path.exists()

# we can use the visualize_h5 function from the command line interface (CLI)
!astrocast visualize-h5 {sample_path.as_posix()}
print("\n")

In [ ]:
# next we load the dataset
io = IO()
data = io.load(sample_path, loc="data/ch0")
print(f"Data shape: {data.shape}, dtype: {data.dtype}, instance type: {type(data)}")

# and visualize the data
video = Video(data)
fig = video.plot_overview()

# uncomment to activate the video player
# viewer = video.show()  

# 1. Preprocessing

In [ ]:
from astrocast.preparation import MotionCorrection, Input, Delta

## 1.1 Data conversion

In [ ]:
test_path = Path("./quickstart.h5")

In [ ]:
# multiple ways exist to convert the data to a format that is compatible with the pipeline
# including .tiff (multi and single page), .avi, .h5 and .czi files
# here we do a simple 'copy' of the sample file

if test_path.exists():
    test_path.unlink()
assert not test_path.exists()

inp = Input()
inp.run(input_path=sample_path, output_path=test_path, loc_in="data/ch0", loc_out="data", compression="gzip",
        chunk_strategy="Z")

!astrocast visualize-h5 {test_path.as_posix()}

# alternatively with CLI:
# !astrocast convert-h5 --output-path {test_path.as_posix()} --h5-loc-in data/ch0 --h5-loc-out data/ch0 --compression gzip --infer-chunks {sample_path.as_posix()}

# 1.2 Motion correction

In [ ]:
MC = MotionCorrection()
MC.run(path=test_path, loc="data/ch0")
MC.save(output=test_path, loc="mc/ch0", chunk_strategy="Z", compression="gzip")

In [ ]:
!astrocast visualize-h5 {test_path.as_posix()}

In [ ]:
# TODO some kind of plotting to show the motion correction!?

## 1.3 Denoising

In [ ]:
from astrocast.denoising import SubFrameDataset, PyTorchNetwork

In [ ]:
# Optional: Download pretrained models
# !astrocast download-models "./models"
# for f in os.listdir("./models"):
#     print(f)

In [ ]:
# Inference dataset
input_size = (128, 128)
loc_in = "mc/ch0"
loc_out = "inf/ch0"
use_cpu = True  # switch to False, if cuda is available

infer_dataset = SubFrameDataset(paths=test_path, loc=loc_in, input_size=input_size, allowed_rotation=0, allowed_flip=-1,
                                shuffle=False, normalize="global", overlap=10, padding="edge")

# Model parameters
model_path = "../denoiser_models/1p_exvivo_input_size_128_128_pre_post_frame_5-gap_frames_0-train_rotation_1_2_3-architecture_3_64_epochs-50.pth"
pre_post_frames = 5
gap_frames = 0
n_stacks, kernels = (3, 64)

net = PyTorchNetwork(infer_dataset, load_model=model_path, val_dataset=None, batch_size=16, n_stacks=n_stacks,
                     kernels=kernels, kernel_size=3, batch_normalize=False,
                     use_cpu=use_cpu)

# Denoise data
net.infer(dataset=infer_dataset, output=test_path, out_loc=loc_out, batch_size=1, dtype=float)

In [ ]:
# !astrocast visualize-h5 {test_path.as_posix()}

In [ ]:
frame = 100
pixels = [(167, 153), (199, 306), (214, 109), (337, 72)]

fig, axx = astroPlot.compare_pixels(pixels=pixels, frame=frame,
                                    data_1=test_path, loc_1="mc/ch0", loc_2="inf/ch0",
                                    title_1="before denoising", title_2="after denoising")

# 1.4 Subtract background

In [ ]:
import astrocast.preparation as prep
import importlib

importlib.reload(prep)

delta = prep.Delta(test_path, loc="inf/ch0")
res_delta = delta.run(method="dF", scale_factor=0.25, neighbors=100, wlen=50, distance=5,
                      max_chunk_size_mb=10, width=5)
delta.save(output_path=test_path, loc="df/ch0", chunk_strategy="balanced", compression="gzip")

# TODO add terminal command

In [ ]:
# !astrocast visualize-h5 {test_path.as_posix()}

In [ ]:
frame = 100
pixels = [(167, 153), (199, 306), (214, 109), (337, 72)]

fig, axx = astroPlot.compare_pixels(pixels=pixels, frame=frame,
                                    data_1=test_path, loc_1="inf/ch0", loc_2="df/ch0",
                                    title_1="before subtraction", title_2="after subtraction")

# 2. Event detection

In [ ]:
from astrocast.detection import Detector
import shutil

In [ ]:
event_path = test_path.with_suffix(".ch0.roi")
if event_path.exists():
    shutil.rmtree(event_path)

detector = Detector(input_path=test_path)
event_dictionary = detector.run(loc="df/ch0", debug=True, exclude_border=20, temporal_prominence=0.3)

In [ ]:
assert event_path.exists()
for f in os.listdir(event_path):
    print(f)

events = Events(event_path)
events

In [ ]:
#!astrocast view-detection-results --video-path {test_path.as_posix()} --loc df/ch0 {event_path.as_posix()}